In [1]:
%pylab inline
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import pickle

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/train.txt.npy'
data_tr = np.load(dataset_tr)
dataset_te = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/test.txt.npy'
data_te = np.load(dataset_te)
vocab = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/vocab.pkl'
vocab = pickle.load(open(vocab,'r'))

In [3]:
vocab_size=len(vocab)
def onehot(data, min_length):
    return np.bincount(data, minlength=min_length)
data_tr = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_tr if np.sum(doc)!=0])
data_te = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_te if np.sum(doc)!=0])
print data_tr.shape
print data_te.shape

(11258, 1995)
(7487, 1995)


In [4]:
n_samples_tr = 11258
n_samples_te = 7488
docs_tr = data_tr
docs_te = data_te

In [5]:
nz=50
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.h_dim = float(network_architecture["n_z"])
        self.a0 = np.ones((1 , 2)).astype(np.float32)/2.0
        self.a1 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        self.a2 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        
        self.mu2a0 = tf.constant((np.log(self.a0).T-np.mean(np.log(self.a0),1)).T)
        self.var2a0 = tf.constant(  ( ( (1.0/self.a0)*( 1 - (2.0/2.0) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a0,1) ).T  )
        
        self.mu2a1 = tf.constant((np.log(self.a1).T-np.mean(np.log(self.a1),1)).T)
        self.var2a1 = tf.constant(  ( ( (1.0/self.a1)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a1,1) ).T  )
        
        self.mu2a2 = tf.constant((np.log(self.a2).T-np.mean(np.log(self.a2),1)).T)
        self.var2a2 = tf.constant(  ( ( (1.0/self.a2)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a2,1) ).T  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''Adding 2 subtopics and 1 super topic.'''
        self.z_mean,self.z_log_sigma_sq = \
            self._recognition_network(self.network_weights["weights_recog"], 
                                      self.network_weights["biases_recog"])
            
        self.z_mean0 = self.z_mean[:,:2]
        self.z_log_sigma_sq0 = self.z_log_sigma_sq[:,:2]
        
#         print self.z_mean0
            
        self.z_mean1 = self.z_mean[:,2:2+nz]
        self.z_log_sigma_sq1 = self.z_log_sigma_sq[:,2:2+nz]
        
        self.z_mean2 = self.z_mean[:,2+nz:]
        self.z_log_sigma_sq2 = self.z_log_sigma_sq[:,2+nz:]

        # Draw one sample z from Gaussian distribution
        '''Adding another noise variable for super toipc: hard coding atm'''
        eps0 = tf.random_normal((1, 2), 0, 1, 
                               dtype=tf.float32)
        n_z = self.network_architecture["n_z"]
        eps1 = tf.random_normal((1, n_z), 0, 1, 
                               dtype=tf.float32)
        eps2 = tf.random_normal((1, n_z), 0, 1, 
                               dtype=tf.float32)
        '''Adding RT for subtopics'''
        self.z0 = tf.add(self.z_mean0, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq0)), eps0))
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z0))
        self.sigma0 = tf.exp(self.z_log_sigma_sq0)
        
        self.z1 = tf.add(self.z_mean1, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq1)), eps1))
        self.z1 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z1))
        
        self.sigma1 = tf.exp(self.z_log_sigma_sq1)
        
        self.z2 = tf.add(self.z_mean2, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq2)), eps2))
        self.z2 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z2))
        self.sigma2 = tf.exp(self.z_log_sigma_sq2)
        
        '''Adding subtopic reconstruction'''
            
#         self.theta = tf.matmul(self.z0,tf.concat(0,[self.z1, self.z2]))
        
        self.theta = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.z0,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.z1,i),0),
                                                                tf.expand_dims(tf.gather(self.z2,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
            
        self.x_reconstr_mean = \
            self._generator_network(tf.nn.dropout(self.theta, self.keep_prob),
                                    self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z):
        all_weights = dict()
        with tf.variable_scope("super"):
            all_weights['weights_recog'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, 2+2*n_z]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, 2+2*n_z])
            }
            all_weights['biases_recog'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([2+2*n_z], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([2+2*n_z], dtype=tf.float32))
            }

        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
        z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                        biases['out_mean']))
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma'])) 
            
        
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
#         self.layer_do_0 = tf.nn.softmax(tf.contrib.layers.batch_norm(z))
        self.layer_do_0 = z
        x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
                                           tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2']))),0.0)
#         x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
#                     tf.matmul(self.layer_do_0, weights['h2']),0.0)))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 

    
        latent_loss0 = 0.5*( tf.reduce_sum(tf.div(self.sigma0,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean0),self.var2a0),
                  (self.mu2a0 - self.z_mean0)),1) - 2 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq0  ,1) ) 
        
        latent_loss1 = 0.5*( tf.reduce_sum(tf.div(self.sigma1,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z_mean1),self.var2a1),
                  (self.mu2a1 - self.z_mean1)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z_log_sigma_sq1  ,1) )
        
        latent_loss2 = 0.5*( tf.reduce_sum(tf.div(self.sigma1,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean2),self.var2a2),
                  (self.mu2a2 - self.z_mean2)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq2  ,1) )

        
        
#         latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
#                                            - tf.square(self.z_mean) 
#                                            - tf.exp(self.z_log_sigma_sq), 1)


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss0 + latent_loss1 + latent_loss2) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.99).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .8})
        return cost,emb
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2 = self.sess.run((self.cost,self.z1,self.z2),
                                        feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0})
        return cost,e1,e2


In [6]:
tf.reset_default_graph()
cost_plot=[]
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch_xs)
#             print cost
#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
#                 sys.exit()

        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)

    return vae,emb

In [20]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=400, # 1st layer encoder neurons
         n_hidden_recog_2=400, # 2nd layer encoder neurons
         n_hidden_gener_1=1995, # 1st layer decoder neurons
         n_input=1995, # MNIST data input (img shape: 28*28)
         n_z=50)  # dimensionality of latent space

batch_size=200
learning_rate=0.01
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
        
minibatches = create_minibatch(docs_tr.astype('float32'))
vae,emb = train(network_architecture, training_epochs=120,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)


'''
Change log: decoder has no dropout or additional tranformations
'''

Learning Rate: 0.01
Epoch: 0001 cost= 2568.726067366
Epoch: 0006 cost= 1523.819786745
Epoch: 0011 cost= 1314.547025240
Epoch: 0016 cost= 1216.470785271
Epoch: 0021 cost= 1124.136862701
Epoch: 0026 cost= 1041.815475055
Epoch: 0031 cost= 941.180747380
Epoch: 0036 cost= 914.609644557
Epoch: 0041 cost= 777.103728625
Epoch: 0046 cost= 734.497924740
Epoch: 0051 cost= 643.157781159
Epoch: 0056 cost= 583.926826405
Epoch: 0061 cost= 498.395218327
Epoch: 0066 cost= 410.934415713
Epoch: 0071 cost= 349.699586999
Epoch: 0076 cost= 271.850347934
Epoch: 0081 cost= 214.188335539
Epoch: 0086 cost= 133.583501754
Epoch: 0091 cost= 76.273801759
Epoch: 0096 cost= 7.806116193
Epoch: 0101 cost= -64.938769778
Epoch: 0106 cost= -117.662790813
Epoch: 0111 cost= -223.958679947
Epoch: 0116 cost= -283.997075929


'\nChange log: decoder has no dropout or additional tranformations\n'

In [21]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb, zip(*sorted(vocab.items(), key=lambda x: x[1]))[0])

francisco committee flight agent papers
game baseball stanley fan hitter
reserve charge eliminate rich order
compound fbi accident fire kid
wings adams stanley playoff rangers
write see time respond seem
much like lot pretty back
islamic israeli arabs arab palestinian
write cd hello setup hus
car thread look accident jason
write thanks appreciate hus anyone
waco batf libertarian liberty clinton
clinton abuse civil government senate
jpeg pixel default gif unix
distribution distribute protocol information mail
go well see kid get
spacecraft technical telephone dc algorithm
warrant clinton nsa crypto bank
dos ide isa scsus scsi
mov bh db myers cs
summer background essentially sector forward
cryptography radar japanese station detector
escrow wiretap clipper encryption crypto
pixel format workstation oname io
write sexual compound reaction waco
islamic libertarian civil arabs terrorist
write hus appreciate spec thanks
go morning floor burn say
improve get back except edition
enforcement se

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=200,max_iter=200, n_jobs=-1,
                                learning_method='online',
                                random_state=0,verbose=1)
%time lda.fit(docs_tr)
beta_sklearn = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]

print_top_words(beta_sklearn, zip(*sorted(vocab.items(), key=lambda x: x[1]))[0])
print lda.perplexity(docs_te)

In [22]:

cost=[]
e1=[]
e2=[]
idx=0
for doc in docs_te:
    idx+=1
    doc = doc.astype('float32')
    n_d = np.sum(doc)
    c,a,b=vae.test(doc)
    cost.append(c/n_d)
    e1.append(a)
    e2.append(b)
    if idx%1000==0:
        print idx
    
# print (np.exp(np.mean(np.array(cost))))
# print np.mean(e1,0)
# print np.mean(e2,0)

1000
2000
3000
4000
5000
6000
7000


In [23]:
print np.argsort(np.mean(e1,0)[0])
print np.argsort(np.mean(e2,0)[0])
print np.mean(e1,0)[0]
print np.mean(e2,0)[0]
print (np.exp(np.mean(np.array(cost))))

[42 20 47 40 28 19 46  2  4  0 33  6 13 32 15 21 48 16 37  5 27 31 44  7 29
 17 39 18 14 23 22 41 25  3 38 49 11  9 35  1 12 30 24 34 45 36 10 26 43  8]
[ 0 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 26 25 24
 23  1  2  3  4  5  6  7  8  9 10 48 11 13 14 15 16 17 18 19 20 21 22 12 49]
[  5.36720455e-03   2.60915067e-02   3.22592864e-03   1.96838174e-02
   3.49952467e-03   1.31452940e-02   7.11442810e-03   1.54265817e-02
   7.25981593e-02   2.05246415e-02   5.95716573e-02   2.02584974e-02
   2.69022435e-02   8.01106822e-03   1.84716806e-02   8.92644376e-03
   1.18768616e-02   1.71108320e-02   1.78014170e-02   1.76525104e-03
   1.90854873e-04   9.57915746e-03   1.87430996e-02   1.85632873e-02
   4.52552363e-02   1.91552043e-02   6.18571751e-02   1.38173094e-02
   1.43716997e-03   1.66011192e-02   3.54726948e-02   1.40461549e-02
   8.64823163e-03   5.64956246e-03   4.58072387e-02   2.55786348e-02
   5.35297617e-02   1.22660380e-02   2.01937575e-02   1.72712449e-02
   8

In [24]:
pickle.dump(cost,open('cost','w'))
pickle.dump(e1,open('e1','w'))
pickle.dump(e2,open('e2','w'))

In [27]:
print np.amax(np.mean(e1,0)[0])

0.0725982
